In [18]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Polygon, Wedge
from ipywidgets import interact
import matplotlib.colors as mcolors

# Silo parameters
density = 90  # lbs/ft³
cone_height = 9.25  # ft
cylinder_height = 34.33  # ft
total_height = cone_height + cylinder_height  # ft
diameter = 11.5  # ft
radius = diameter / 2  # ft
truck_capacity = 55000  # lbs

# Sensor setup
sensor_start_ft = 3.58  # Sensor starts 3.58 ft above the bottom of the cone
sensor_cone_height_ft = 5.67  # Sensor portion in the cone
sensor_cylinder_height_ft = 28.5  # Sensor portion in the cylinder
sensor_total_height_ft = sensor_cone_height_ft + sensor_cylinder_height_ft  # Total sensor range (approx. 34.17 ft)
sensor_end_ft = sensor_start_ft + sensor_total_height_ft  # Sensor end position

# Fixed values
below_sensor_weight = 11155.52  # lbs, weight of material below the sensor
total_silo_capacity = 349775  # lbs, total weight capacity of the silo when fully loaded

# Function to calculate total volume up to a given height from the bottom
def calculate_total_volume_up_to_height(height):
    if height <= 0:
        return 0
    elif height <= cone_height:
        return (1/3) * np.pi * radius**2 * height
    else:
        cone_volume = (1/3) * np.pi * radius**2 * cone_height
        cylinder_volume = np.pi * radius**2 * (height - cone_height)
        return cone_volume + cylinder_volume

# Function to calculate volume and weight above the sensor
def calculate_volume_weight(level_percentage):
    filled_height = sensor_start_ft + (level_percentage / 100) * sensor_total_height_ft
    total_volume_to_fill = calculate_total_volume_up_to_height(filled_height)
    total_volume_to_sensor_start = calculate_total_volume_up_to_height(sensor_start_ft)
    volume_above_sensor = total_volume_to_fill - total_volume_to_sensor_start
    weight_above_sensor = volume_above_sensor * density
    trucks_needed = weight_above_sensor / truck_capacity
    return filled_height, volume_above_sensor, weight_above_sensor, trucks_needed

# Function to create a gradient fill color based on level percentage
def get_gradient_color(level_percentage):
    cmap = plt.cm.get_cmap("RdYlGn_r")  # Red to Green gradient
    return cmap(level_percentage / 100)

# Function to plot the silo with gradient fill and a gauge indicator
def plot_silo(level_percentage):
    filled_height, volume_above_sensor, weight_above_sensor, trucks_needed = calculate_volume_weight(level_percentage)

    # Calculate sensor level based on percentage input (0% to 100%)
    sensor_level = (level_percentage / 100) * sensor_total_height_ft  # Scale sensor level for display

    fig, (ax_silo, ax_gauge) = plt.subplots(1, 2, figsize=(8, 10), gridspec_kw={'width_ratios': [3, 1]})
    ax_silo.set_xlim(-radius * 1.5, radius * 2)
    ax_silo.set_ylim(0, total_height + 5)
    ax_silo.set_aspect('equal')

    # Draw silo
    cone = Polygon([(-radius, cone_height), (0, 0), (radius, cone_height)], closed=True, edgecolor='black', facecolor='lightgrey')
    ax_silo.add_patch(cone)
    cylinder = Rectangle((-radius, cone_height), 2 * radius, cylinder_height, edgecolor='black', facecolor='darkgrey')
    ax_silo.add_patch(cylinder)

    # Sensor range
    ax_silo.plot([0, 0], [sensor_start_ft, sensor_end_ft], color='orange', linestyle='--', linewidth=2, label='Sensor Range')
    ax_silo.plot([0], [sensor_start_ft], marker='o', color='orange', label='Sensor Start (0%)')
    ax_silo.plot([0], [sensor_end_ft], marker='o', color='orange', label='Sensor Full (100%)')

    # Gradient fill
    fill_color = get_gradient_color(level_percentage)
    if filled_height <= cone_height:
        bottom_radius = (sensor_start_ft / cone_height) * radius
        top_radius = (filled_height / cone_height) * radius
        fill_cone = Polygon([
            (-bottom_radius, sensor_start_ft),
            (0, 0),
            (bottom_radius, sensor_start_ft),
            (top_radius, filled_height),
            (-top_radius, filled_height)
        ], closed=True, edgecolor='blue', facecolor=fill_color, alpha=0.6)
        ax_silo.add_patch(fill_cone)
    else:
        if sensor_start_ft < cone_height:
            bottom_radius = (sensor_start_ft / cone_height) * radius
            fill_cone = Polygon([
                (-bottom_radius, sensor_start_ft),
                (0, 0),
                (bottom_radius, sensor_start_ft),
                (radius, cone_height),
                (-radius, cone_height)
            ], closed=True, edgecolor='blue', facecolor=fill_color, alpha=0.6)
            ax_silo.add_patch(fill_cone)
            fill_cylinder_start = cone_height
        else:
            fill_cylinder_start = sensor_start_ft
        fill_cylinder_height = filled_height - fill_cylinder_start
        fill_cylinder = Rectangle((-radius, fill_cylinder_start), 2 * radius, fill_cylinder_height, edgecolor='blue', facecolor=fill_color, alpha=0.6)
        ax_silo.add_patch(fill_cylinder)

    # Display silo information
    text_x = radius * 1.1
    ax_silo.text(text_x, total_height + 4, f"Fill Level: {level_percentage}%", fontsize=10, ha='left')
    ax_silo.text(text_x, total_height + 3, f"Volume Above Sensor: {volume_above_sensor:.2f} ft³", fontsize=10, ha='left')
    ax_silo.text(text_x, total_height + 2, f"Weight Above Sensor: {weight_above_sensor:.2f} lbs", fontsize=10, ha='left')
    ax_silo.text(text_x, total_height + 1, f"Trucks Needed: {trucks_needed:.2f}", fontsize=10, ha='left')
    ax_silo.text(text_x, total_height, f"Below Sensor Weight: {below_sensor_weight:.2f} lbs", fontsize=10, ha='left', color='darkred')
    ax_silo.text(text_x, total_height - 1, f"Total Silo Capacity: {total_silo_capacity} lbs", fontsize=10, ha='left', color='darkgreen')
    ax_silo.text(text_x, total_height - 2, f"Sensor Level: {sensor_level:.2f} ft", fontsize=10, ha='left', color='blue')  # Sensor Level

    # Gauge for fill level
    ax_gauge.axis("equal")
    ax_gauge.add_patch(Wedge((0, 0), 1, 0, 180, color="lightgrey", alpha=0.3))
    ax_gauge.add_patch(Wedge((0, 0), 1, 0, level_percentage * 1.8, color=get_gradient_color(level_percentage)))
    ax_gauge.text(0, 0, f"{level_percentage}%", ha="center", va="center", fontsize=16, color="black")
    ax_gauge.set_xlim(-1.2, 1.2)
    ax_gauge.set_ylim(-0.2, 1.2)
    ax_gauge.axis("off")

    # Final touches
    ax_silo.legend(loc='lower right', bbox_to_anchor=(1.8, 0.1))
    ax_silo.axis('off')
    ax_silo.set_title("Silo Fill Level Visualization", fontsize=14)
    plt.show()

# Interactive slider for fill level percentage
interact(plot_silo, level_percentage=(0, 100, 1))


interactive(children=(IntSlider(value=50, description='level_percentage'), Output()), _dom_classes=('widget-in…

<function __main__.plot_silo(level_percentage)>